In [32]:
import xlrd
import os
from pandas import Series,DataFrame
import numpy as np
import pandas as pd

In [33]:
file_name='data/convert_2.xlsx'
read_data=pd.read_excel(file_name,sheetname=0)

In [34]:
read_data.head()

,年 月 日,风暴潮组,海浪组,海浪组.1,海浪组.2,海冰组,海冰组.1
0,NaT,NaN,主班,副班,警报班,主班,副班
1,2017-12-01,刘秋兴,韦锋余,徐瑞,王娟娟,唐茂宁,刘琼
2,2017-12-02,刘秋兴,邢闯,韦锋余,徐瑞,唐茂宁,刘琼
3,2017-12-03,刘秋兴,侯放,邢闯,韦锋余,唐茂宁,刘琼
4,2017-12-04,梁森栋,高志一,侯放,邢闯,唐茂宁,刘琼


* 读取行标题

In [26]:
read_data.columns.tolist()

['年  月  日', '风暴潮组', '海浪组', 'Unnamed: 3', 'Unnamed: 4', '海冰组', 'Unnamed: 6']

* 读取第一行

In [31]:
# 会出错 KeyError: 1
# read_data[1]
read_data[:1]

,年 月 日,风暴潮组,海浪组,Unnamed: 3,Unnamed: 4,海冰组,Unnamed: 6
0,NaT,NaN,主班,副班,警报班,主班,副班


#### 转换日期

In [20]:
import time

In [21]:
time.gmtime(43072.0)

time.struct_time(tm_year=1970, tm_mon=1, tm_mday=1, tm_hour=11, tm_min=57, tm_sec=52, tm_wday=3, tm_yday=1, tm_isdst=0)

In [22]:
targetdate=86640+43072.0
targetdate

129712.0

In [23]:
time.gmtime(targetdate)

time.struct_time(tm_year=1970, tm_mon=1, tm_mday=2, tm_hour=12, tm_min=1, tm_sec=52, tm_wday=4, tm_yday=2, tm_isdst=0)